In [1]:
from taxcalc import *
import pandas as pd
import numpy as np
from bokeh.io import show, output_notebook, output_file, save
from bokeh.models import ColumnDataSource, NumeralTickFormatter, Span, HoverTool, Panel, Tabs, CustomJS, Title
from bokeh.models.widgets import Dropdown, Select
from bokeh.plotting import figure
from bokeh.transform import dodge
from bokeh.layouts import column, row, WidgetBox, layout

In [2]:
recs = Records()
pol = Policy()
calc_base = Calculator(records=recs, policy=pol)

In [3]:
calc_base.advance_to_year(2020)
calc_base.calc_all()

# eitc_base = calc.weighted_total('eitc')
# ctc_base = calc.weighted_total('c07220') + calc.weighted_total('c11070')
# eitc_base / 1e9

In [4]:
pol._EITC_c[7]

array([ 540.5 , 3604.35, 5952.74, 6697.35])

In [5]:
ratio_1 = pol._EITC_c[7][1] / pol._EITC_c[7][0]
ratio_2 = pol._EITC_c[7][2] / pol._EITC_c[7][0]
ratio_3 = pol._EITC_c[7][3] / pol._EITC_c[7][0]

In [6]:
# Replace all existing child tax credits with an expanded EITC on a budget neutral basis

eitc = 1679
pol_eitc = Policy()
eitc_ref = {'EITC_c': {2020: [eitc, float(eitc * ratio_1), float(eitc * ratio_2), float(eitc * ratio_3)]},
            'CTC_c': {2020: 0.0},
            'ACTC_c': {2020: 0.0}}
pol_eitc.implement_reform(eitc_ref)

calc_eitc = Calculator(records=recs, policy=pol_eitc)
calc_eitc.advance_to_year(2020)
calc_eitc.calc_all()
(calc_base.weighted_total('iitax') - calc_eitc.weighted_total('iitax')) / 1e9

CTC_c was redefined in release 1.0.0



0.015336960344482422

In [7]:
eitc, float(eitc * ratio_1), float(eitc * ratio_2), float(eitc * ratio_3)

(1679, 11196.491489361702, 18491.490212765955, 20804.534042553194)

In [8]:
# Replace EITC with a fully refundable CTC (to do this, eliminate the existing CTC and ACTC completely and use "new refundable child tax credit").

ctc_new = 2695
pol_ctc_refund = Policy()
ctc_refund = {'EITC_c': {2020: [0,0,0,0]},
            'CTC_c': {2020: 0.0},
            'ACTC_c': {2020: 0.0},
            'CTC_new_c': {2020: ctc_new},
            'CTC_new_ps': {2020: [200000.0, 400000.0, 200000.0, 200000.0, 400000.0]},
            'CTC_new_prt': {2020: 0.05},
            'CTC_new_for_all': {2020: True}
           }
pol_ctc_refund.implement_reform(ctc_refund)

calc_ctc_refund = Calculator(records=recs, policy=pol_ctc_refund)
calc_ctc_refund.advance_to_year(2020)
calc_ctc_refund.calc_all()
(calc_base.weighted_total('iitax') - calc_ctc_refund.weighted_total('iitax')) / 1e9

CTC_c was redefined in release 1.0.0



0.031319975778564456

In [9]:
# Replace EITC with a CTC refundable up to the payroll tax.

ctc_fica_amt = 3292
pol_ctc_fica = Policy()
ctc_fica = {'EITC_c': {2020: [0,0,0,0]},
            'CTC_c': {2020: 0.0},
            'ACTC_c': {2020: 0.0},
            'CTC_new_c': {2020: ctc_fica_amt},
            'CTC_new_ps': {2020: [200000.0, 400000.0, 200000.0, 200000.0, 400000.0]},
            'CTC_new_prt': {2020: 0.05},
            'CTC_new_for_all': {2020: True},
            'CTC_new_refund_limited': {2020: True},
            'CTC_new_refund_limit_payroll_rt': {2020: 1.0},
            'CTC_new_refund_limited_all_payroll': {2020: True}
           }
pol_ctc_fica.implement_reform(ctc_fica)

calc_ctc_fica = Calculator(records=recs, policy=pol_ctc_fica)
calc_ctc_fica.advance_to_year(2020)
calc_ctc_fica.calc_all()
(calc_base.weighted_total('iitax') - calc_ctc_fica.weighted_total('iitax')) / 1e9

CTC_c was redefined in release 1.0.0



-0.0005017335256347656

In [10]:
# Replace EITC with a non-refundable CTC.

ctc_nonrefund = 7114
pol_ctc = Policy()
ctc_nf = {'EITC_c': {2020: [0,0,0,0]},
           'CTC_c': {2020: ctc_nonrefund},
           'ACTC_c': {2020: 0}}
pol_ctc.implement_reform(ctc_nf)
calc_ctc_nonrefund = Calculator(records=recs, policy=pol_ctc)
calc_ctc_nonrefund.advance_to_year(2020)
calc_ctc_nonrefund.calc_all()
(calc_base.weighted_total('iitax') - calc_ctc_nonrefund.weighted_total('iitax')) / 1e9

CTC_c was redefined in release 1.0.0



0.0010630128818359375

In [11]:
# create subsetted datasets by marital status and num dependents

VARS = ['MARS', 'n24', 'expanded_income', 'c00100', 'aftertax_income',
        'iitax', 'payrolltax', 'combined', 's006', 'XTOT',
        'ubi', 'benefit_cost_total', 'benefit_value_total']

BINS = ['$<10K', '$10-20K', '$20-30K', '$30-40K', '$40-50K',
       '$50-75K', '$75-100K', '$100-200K', '$200-500K', '>$500K', 'ALL']

def make_data(calc, mars, deps):
    df_reform = calc.dataframe(VARS)
    df_base = calc_base.dataframe(VARS)

    if mars =='all' and deps=='all':
        df_filter_reform = df_reform
        df_filter_base = df_base
    elif mars == 'all' and deps is not 'all':
        df_filter_reform = df_reform.loc[(df_reform['n24'] == deps)]
        df_filter_base = df_base.loc[(df_reform['n24'] == deps)]
    elif deps == 'all' and mars is not 'all':
        df_filter_reform = df_reform.loc[(df_reform['MARS'] == mars)]
        df_filter_base = df_base.loc[(df_reform['MARS'] == mars)]
    else:
        df_filter_reform = df_reform.loc[(df_reform['MARS'] == mars) & (df_reform['n24'] == deps)]
        df_filter_base = df_base.loc[(df_base['MARS'] == mars) & (df_base['n24'] == deps)]
    
    diff_table = create_difference_table(df_filter_base, df_filter_reform, 'custom_income_bins', 'iitax')
    diff = pd.DataFrame()
    diff['count'] = diff_table['count']
    diff['perc_inc'] = diff_table['perc_inc'] / 100
    diff['perc_cut'] = diff_table['perc_cut'] / 100
    diff['mean'] = diff_table['mean']
    diff['pc_aftertaxinc'] = diff_table['pc_aftertaxinc'] / 100
    diff['bins'] = BINS
    return ColumnDataSource(diff)

In [12]:
ref1_mar_all = make_data(calc_eitc, 2, 'all')
ref1_sin_all = make_data(calc_eitc, 1, 'all')
ref1_all_all = make_data(calc_eitc, 'all', 'all')
ref1_all_0 = make_data(calc_eitc, 'all', 0)
ref1_all_1 = make_data(calc_eitc, 'all', 1)
ref1_all_2 = make_data(calc_eitc, 'all', 2)
ref1_all_3 = make_data(calc_eitc, 'all', 3)

ref2_mar_all = make_data(calc_ctc_refund, 2, 'all')
ref2_sin_all = make_data(calc_ctc_refund, 1, 'all')
ref2_all_all = make_data(calc_ctc_refund, 'all', 'all')
ref2_all_0 = make_data(calc_ctc_refund, 'all', 0)
ref2_all_1 = make_data(calc_ctc_refund, 'all', 1)
ref2_all_2 = make_data(calc_ctc_refund, 'all', 2)
ref2_all_3 = make_data(calc_ctc_refund, 'all', 3)

ref3_mar_all = make_data(calc_ctc_fica, 2, 'all')
ref3_sin_all = make_data(calc_ctc_fica, 1, 'all')
ref3_all_all = make_data(calc_ctc_fica, 'all', 'all')
ref3_all_0 = make_data(calc_ctc_fica, 'all', 0)
ref3_all_1 = make_data(calc_ctc_fica, 'all', 1)
ref3_all_2 = make_data(calc_ctc_fica, 'all', 2)
ref3_all_3 = make_data(calc_ctc_fica, 'all', 3)

ref4_mar_all = make_data(calc_ctc_nonrefund, 2, 'all')
ref4_sin_all = make_data(calc_ctc_nonrefund, 1, 'all')
ref4_all_all = make_data(calc_ctc_nonrefund, 'all', 'all')
ref4_all_0 = make_data(calc_ctc_nonrefund, 'all', 0)
ref4_all_1 = make_data(calc_ctc_nonrefund, 'all', 1)
ref4_all_2 = make_data(calc_ctc_nonrefund, 'all', 2)
ref4_all_3 = make_data(calc_ctc_nonrefund, 'all', 3)

In [13]:
# output_notebook()

bins_graph = ['$<10K', '$10-20K', '$20-30K', '$30-40K', '$40-50K',
       '$50-75K', '$75-100K', '$100-200K', '$200-500K', '>$500K']

In [14]:
# def pc_change(src, title):
#     p = figure(x_range=bins_graph, plot_height=300, plot_width=750, tools="", title=title)
#     p.vbar(x='bins', top='mean', width=0.5, source=src, color='#ff7f0e', alpha=0.7)
    
#     hline = Span(location=0, dimension='width', line_color='gray')
#     p.renderers.extend([hline])

#     p.yaxis.formatter = NumeralTickFormatter(format='$0,000')
#     p.yaxis.axis_label = 'Mean Tax Increase (Cut)'
#     p.xaxis.axis_label = 'AGI Bin'
#     p.xgrid.grid_line_color = None
#     p.yaxis.minor_tick_line_color = None

#     hover = HoverTool(tooltips='@mean{$0,000}')
#     p.add_tools(hover)
#     return p

In [15]:
# title1 = "Replace all existing child tax credits with an expanded EITC on a budget neutral basis"
# ref1_all_deps = Panel(child=pc_change(ref1_all_all, title1), title="All Units")
# ref1_sin = Panel(child=pc_change(ref1_sin_all, title1), title="Single")
# ref1_mar = Panel(child=pc_change(ref1_mar_all, title1), title="Married")
# ref1_0 = Panel(child=pc_change(ref1_all_0, title1), title="0 Kids")
# ref1_1 = Panel(child=pc_change(ref1_all_1, title1), title="1 Kid")
# ref1_2 = Panel(child=pc_change(ref1_all_2, title1), title="2 Kids")
# ref1_3 = Panel(child=pc_change(ref1_all_3, title1), title="3 Kids")

# ref1_tabs = Tabs(tabs=[ref1_all_deps, ref1_sin, ref1_mar, ref1_0, ref1_1, ref1_2, ref1_3])
# show(ref1_tabs)

# output_file('reform1.html')
# save(ref1_tabs)

In [16]:
# title2 = "Replace EITC with a fully refundable CTC"
# ref2_all_deps = Panel(child=pc_change(ref2_all_all, title2), title="All Units")
# ref2_sin = Panel(child=pc_change(ref2_sin_all, title2), title="Single")
# ref2_mar = Panel(child=pc_change(ref2_mar_all, title2), title="Married")
# ref2_0 = Panel(child=pc_change(ref2_all_0, title2), title="0 Kids")
# ref2_1 = Panel(child=pc_change(ref2_all_1, title2), title="1 Kid")
# ref2_2 = Panel(child=pc_change(ref2_all_2, title2), title="2 Kids")
# ref2_3 = Panel(child=pc_change(ref2_all_3, title2), title="3 Kids")

# ref2_tabs = Tabs(tabs=[ref2_all_deps, ref2_sin, ref2_mar, ref2_0, ref2_1, ref2_2, ref2_3])
# show(ref2_tabs)
# output_file('reform2.html')
# save(ref2_tabs)

In [17]:
# title3 = "Replace EITC with a CTC refundable up to the payroll tax"
# ref3_all_deps = Panel(child=pc_change(ref3_all_all, title3), title="All Units")
# ref3_sin = Panel(child=pc_change(ref3_sin_all, title3), title="Single")
# ref3_mar = Panel(child=pc_change(ref3_mar_all, title3), title="Married")
# ref3_0 = Panel(child=pc_change(ref3_all_0, title3), title="0 Kids")
# ref3_1 = Panel(child=pc_change(ref3_all_1, title3), title="1 Kid")
# ref3_2 = Panel(child=pc_change(ref3_all_2, title3), title="2 Kids")
# ref3_3 = Panel(child=pc_change(ref3_all_3, title3), title="3 Kids")

# ref3_tabs = Tabs(tabs=[ref3_all_deps, ref3_sin, ref3_mar, ref3_0, ref3_1, ref3_2, ref3_3])
# show(ref3_tabs)

# output_file('reform3.html')
# save(ref3_tabs)

In [18]:
# title4 = "Replace EITC with a non-refundable CTC"
# ref4_all_deps = Panel(child=pc_change(ref4_all_all, title4), title="All Units")
# ref4_sin = Panel(child=pc_change(ref4_sin_all, title4), title="Single")
# ref4_mar = Panel(child=pc_change(ref4_mar_all, title4), title="Married")
# ref4_0 = Panel(child=pc_change(ref4_all_0, title4), title="0 Kids")
# ref4_1 = Panel(child=pc_change(ref4_all_1, title4), title="1 Kid")
# ref4_2 = Panel(child=pc_change(ref4_all_2, title4), title="2 Kids")
# ref4_3 = Panel(child=pc_change(ref4_all_3, title4), title="3 Kids")

# ref4_tabs = Tabs(tabs=[ref4_all_deps, ref4_sin, ref4_mar, ref4_0, ref4_1, ref4_2, ref4_3])
# show(ref4_tabs)

# output_file('reform4.html')
# save(ref4_tabs)

In [19]:
def mean_change_drop(src1, src2, src3, src4):
    p = figure(x_range=bins_graph, plot_height=300, plot_width=750, tools="", title=" ")
    ref1 = p.vbar(x='bins', top='mean', width=0.5, source=src1, color='#ff7f0e', alpha=0.7)
    ref2 = p.vbar(x='bins', top='mean', width=0.5, source=src2, color='#ff7f0e', alpha=0.7)
    ref3 = p.vbar(x='bins', top='mean', width=0.5, source=src3, color='#ff7f0e', alpha=0.7)
    ref4 = p.vbar(x='bins', top='mean', width=0.5, source=src4, color='#ff7f0e', alpha=0.7)
    
    ref1.visible = False
    ref2.visible = False
    ref3.visible = False
    ref4.visible = False
    
    hline = Span(location=0, dimension='width', line_color='gray')
    p.renderers.extend([hline])

    p.yaxis.formatter = NumeralTickFormatter(format='$0,000')
    p.yaxis.axis_label = 'Mean Tax Increase (Cut)'
    p.xaxis.axis_label = 'AGI Bin'
    p.xgrid.grid_line_color = None
    p.yaxis.minor_tick_line_color = None
    
    hover = HoverTool(tooltips='@mean{$0,000}')
    p.add_tools(hover)
    
    plot_js = """
    if (select.value == 'ref1') {
        p.title.text = 'Replace all existing child tax credits with an expanded EITC on a budget neutral basis';
        obj1.visible = true;
        obj2.visible = false;
        obj3.visible = false;
        obj4.visible = false;
    } else if (select.value == 'ref2') {
        p.title.text = 'Replace EITC with a fully refundable CTC';
        obj1.visible = false;
        obj2.visible = true;
        obj3.visible = false;
        obj4.visible = false;
    } else if (select.value == 'ref3') {
        p.title.text = 'Replace EITC with a CTC refundable up to the payroll tax';
        obj1.visible = false;
        obj2.visible = false;
        obj3.visible = true;
        obj4.visible = false;
    } else if (select.value == 'ref4') {
        p.title.text = 'Replace EITC with a non-refundable CTC';
        obj1.visible = false;
        obj2.visible = false;
        obj3.visible = false;
        obj4.visible = true;
    }
    else if (select.value == 'none') {
        p.title.text = '';
        obj1.visible = false;
        obj2.visible = false;
        obj3.visible = false;
        obj4.visible = false;
    }
    """
    
    select = Select(title="", value="none", options=[("none", "Choose Reform"), ("ref1", "Reform A"), ("ref2", "Reform B"), ("ref3", "Reform C"), ("ref4", "Reform D")])
    callback = CustomJS(code=plot_js, args={"p": p, "select": select, "obj1": ref1, "obj2": ref2, "obj3": ref3, "obj4": ref4})
    select.js_on_change('value', callback)
    layout = column(p, select)
    return layout

# all_mstat = Panel(child=mean_change_drop(ref1_all_all, ref2_all_all, ref3_all_all, ref4_all_all), title="All Units")
# single = Panel(child=mean_change_drop(ref1_sin_all, ref2_sin_all, ref3_sin_all, ref4_sin_all), title="Single")
# joint = Panel(child=mean_change_drop(ref1_mar_all, ref2_mar_all, ref3_mar_all, ref4_mar_all), title="Married")

# mstat_tabs = Tabs(tabs=[all_mstat, single, joint])

# # show(mstat_tabs)
# output_file('mstat.html')
# save(mstat_tabs)

In [20]:
all_deps = Panel(child=mean_change_drop(ref1_all_all, ref2_all_all, ref3_all_all, ref4_all_all), title="All Units")
deps_0 = Panel(child=mean_change_drop(ref1_all_0, ref2_all_0, ref3_all_0, ref4_all_0), title="No Kids")
deps_1 = Panel(child=mean_change_drop(ref1_all_1, ref2_all_1, ref3_all_1, ref4_all_1), title="1 Kid")
deps_2 = Panel(child=mean_change_drop(ref1_all_2, ref2_all_2, ref3_all_2, ref4_all_2), title="2 Kids")
deps_3 = Panel(child=mean_change_drop(ref1_all_3, ref2_all_3, ref3_all_3, ref4_all_3), title="3 Kids")

deps_tabs = Tabs(tabs=[all_deps, deps_0, deps_1, deps_2, deps_3])

# show(deps_tabs)
output_file('deps.html')
save(deps_tabs)

'/Users/petermetz/Tax-Calculator/deps.html'